In [ ]:
#default_exp synchro.processing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import numpy as np

In [ ]:
#export
def get_thresholds(data):
    max_val = max(data[int(len(data)/2):int(len(data)*3/4)]) #Looking for a max in a quarter of the data, from the middle
    high_thresh = max_val*3/4 # High threshold set at 3/4th of the max
    low_thresh  = max_val*1/4
    return low_thresh, high_thresh

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted 02_processing.ipynb.
Converted 03_modelling.ipynb.
Converted 04_plotting.ipynb.
Converted 05_database.ipynb.
Converted 10_synchro.io.ipynb.
Converted 11_synchro.extracting.ipynb.
Converted 12_synchro.processing.ipynb.
Converted 99_testdata.ipynb.
Converted index.ipynb.
